# TensorBoard Guide
written by [Mattia Chiari](mailto:m.chiari017@unibs.it)



## Sources:
- https://www.tensorflow.org/tensorboard/get_started
- https://neptune.ai/blog/tensorboard-tutorial

## Install TensorBoard

- Anaconda 

```python
conda install -c conda-forge tensorboard
```

- pip

```
pip install tensorboard
```

- Colab (magic command)

```
%load_ext tensorboard
```

### Import TensorBoard for Colab

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Imports

In [ ]:
import os
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import io
from sklearn import metrics

## TensorBoard callback

Keras callbacks are action that are performed during the execution of the model `fit` method. 

TensorBoard callback is responsible for  logging events such as Activation Histograms, Metrics Summary Plots, Profiling and Training Graph Visualizations.

The TensorBoard callback also takes other parameters:
- **log_dir** is the path to the directory where to save the logs
- **histogram_freq** is the frequency at which to compute activation and weight histograms for layers of the model. Setting this to 0 means that histograms will not be computed. In order for this to work you have to set the validation data or the validation split. 
- **write_graph** dictates if the graph will be visualized in TensorBoard
- **write_images** when set to true, model weights are visualized as an image in TensorBoard
- **update_freq** determines how losses and metrics are written to TensorBoard. When set to an integer, say 100, losses and metrics are logged every 100 batches. When set to batch the losses and metrics are set after every batch. When set to epoch they are written after every epoch
- **profile_batch** determines which batches will be profiled. By default, the second batch is profiled. You can also set, for example from 5 and to 10, to profile batches 5 to 10, i.e profile_batch=’5,10′ . Setting profile_batch to 0 disables profiling.
- **embeddings_freq** the frequency at which the embedding layers will be visualized. Setting this to zero means that the embeddings will not be visualized


### Callback example

In [ ]:
# Set the log dir 
log_folder = 'logs'

# Define the callback
tb_callback = TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)

## Dataset: FashonMNIST

You can find more information about the dataset [here](https://github.com/zalandoresearch/fashion-mnist)

### Constants

In [ ]:
# FashonMNIST labels values
class_names = {0:	'T-shirt/top',
               1:	'Trouser',
               2:	'Pullover',
               3:	'Dress',
               4:	'Coat',
               5:	'Sandal',
               6:	'Shirt',
               7:	'Sneaker',
               8:	'Bag',
               9:	'Ankle boot'}

### Custom functions

In [ ]:
def image_grid(x: list, y: list, figures: int = 36, cols: int = 6):
    """
    Plot a grid of images

    Args:
        x (list): list of images
        y (list): list of labels as integers
        figures (int, optional): number of figures to plot. Defaults to 36.
        cols (int, optional): number of columns in the grid. Defaults to 6.

    Raises:
        ValueError: if x and y have different lengths

    Returns:
        matplotlib.figure.Figure: a figure with a grid of images
    """
    if len(x) != len(y):
        raise ValueError("x and y must have the same length")

    figure = plt.figure(figsize=(12,12))

    lines = np.ceil(float(figures)/cols)
    for i in range(figures):
        plt.subplot(lines, cols, i + 1)
        plt.xlabel(class_names[y[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(x[i], cmap=plt.cm.coolwarm)
        #plt.tight_layout()

    return figure

### Code

In [ ]:
# Download FashonMNIST
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()



In [ ]:
figure_train = image_grid(x_train, y_train)

In [ ]:
figure_test = image_grid(x_test, y_test)

## Run TensorBoard

We will use the [FashonMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset to train a simple model.

### Custom functions

In [ ]:
def create_model():
  """
  Create a simple sequential model

  Returns:
      Model: a simple sequential model 
  """
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [ ]:
def train_model(x_train: np.ndarray,
                y_train: np.ndarray,
                x_test: np.ndarray,
                y_test: np.ndarray,
                log_folder: str = None,
                epochs: int = 5):
  """
  Train a simple sequential model
  
  Args:
      x_train (np.ndarray): training data
      y_train (np.ndarray): training labels
      x_test (np.ndarray): test data
      y_test (np.ndarray): test labels
      log_folder (str, optional): directory to save logs. Defaults to './logs/'.
      epochs (int, optional): number of epochs to train. Defaults to 5. 
  """
  # Create and train the model
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  # Define log dir
  if log_folder == None:
    logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  else:
    logdir = log_folder

  # Create callbacks list
  callbacks = [tf.keras.callbacks.TensorBoard(logdir, 
                                              histogram_freq=1, 
                                              profile_batch='250,500')]

  # Train the model
  model.fit(x=x_train, 
            y=y_train, 
            epochs=epochs, 
            validation_data=(x_test, y_test), 
            callbacks=callbacks)

### Code

In [ ]:
# Run tensorboard
%tensorboard --logdir=logs

In [ ]:
# Scale the images
x_train, x_test = x_train/255.0, x_test/255.0

# Create and train the model
train_model(x_train, y_train, x_test, y_test)

In [ ]:
!kill 111
!rm -r './logs'

## Tensorboard Images
Apart from visualizing image tensors, you can also visualize actual images in TensorBoard. In order to illustrate that,  you need to convert the MNIST tensors to images using Matplotlib. After that, you need to use `tf.summary.image` to plot the images in Tensorboard. 



### Custom functions

In [ ]:
def plot_to_image(figure: plt.figure):
    """
    Convert a matplotlib figure to a tensor

    Args:
        figure (plt.figure): a matplotlib figure

    Returns:
        tensor: a tensor that contains the image
    """
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

In [ ]:
def plot_confusion_matrix(cm: np.ndarray, class_names: list, show_percentage: bool = False): 
    """
    Plot a confusion matrix

    Args:
        cm (np.ndarray): a confusion matrix
        class_names (list): a list of class names
        show_percentage (bool): if True, show the percentage of each class. 
                                Defaults to False.

    Returns:
        matplotlib.figure.Figure: a figure with a confusion matrix
    """
    figure = plt.figure(figsize=(8, 8)) 
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent) 
    plt.title("Confusion matrix") 
    plt.colorbar() 
    tick_marks = np.arange(len(class_names)) 
    plt.xticks(tick_marks, class_names, rotation=45) 
    plt.yticks(tick_marks, class_names)

    if show_percentage:
      cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)  
    threshold = cm.max() / 2. 
  
    for i in range(cm.shape[0]):
      for j in range(cm.shape[1]): 
          color = "white" if cm[i, j] > threshold else "black"   
          plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)  
    
    plt.tight_layout() 
    plt.ylabel('True label') 
    plt.xlabel('Predicted label') 

    return figure

In [ ]:
def create_cm_callback(model: tf.keras.Model, x_test:np.ndarray, y_test: np.ndarray, class_names: list, file_writer_cm: tf.summary.SummaryWriter):
    """
    Create a callback that will plot a confusion matrix

    Args:
        model (tf.keras.Model): the model
        x_test (np.ndarray): test data
        y_test (np.ndarray): test labels
        class_names (list): a list of class names
        file_writer_cm (tf.summary.SummaryWriter): a file writer
    """

    def log_confusion_matrix(epoch: int, logs: dict):
        """
        Log a confusion matrix

        Args:
            epoch (int): current epoch
            logs (dict): logs
        """
        predictions = model.predict(x_test)
        predictions = np.argmax(predictions, axis=1)

        cm = metrics.confusion_matrix(y_test, predictions)
        figure = plot_confusion_matrix(cm, class_names=class_names)
        cm_image = plot_to_image(figure)
        
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    return log_confusion_matrix

In [ ]:
def train_model_with_cm(x_train: np.ndarray,
                        y_train: np.ndarray,
                        x_test: np.ndarray,
                        y_test: np.ndarray,
                        log_folder: str = None,
                        epochs: int = 5):
  """
  Train a simple sequential model
  
  Args:
      x_train (np.ndarray): training data
      y_train (np.ndarray): training labels
      x_test (np.ndarray): test data
      y_test (np.ndarray): test labels
      log_folder (str, optional): directory to save logs. Defaults to './logs/'.
      epochs (int, optional): number of epochs to train. Defaults to 5. 
  """
  # Create and train the model
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  # Define log dir
  if log_folder == None:
    logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  else:
    logdir = log_folder


  # Create file writer
  file_writer = tf.summary.create_file_writer(logdir)

  # Create callbacks list
  callbacks = [tf.keras.callbacks.TensorBoard(logdir, 
                                              histogram_freq=1, 
                                              profile_batch='250,500'),
               tf.keras.callbacks.LambdaCallback(on_epoch_end=
                                                 create_cm_callback(model,
                                                                    x_test,
                                                                    y_test,
                                                                    class_names.values(),
                                                                    file_writer))]

  # Train the model
  model.fit(x=x_train, 
            y=y_train, 
            epochs=epochs, 
            validation_data=(x_test, y_test), 
            callbacks=callbacks)


### Code

#### Static images

In [ ]:
# Copy log folder path
logdir = os.path.join('/content/logs/20221010-092859', 'plots')


# Create a file writer
file_writer = tf.summary.create_file_writer(logdir)

# Save the image

img_name = 'FashonMNIST example'
figure = image_grid(x_train, y_train)
with file_writer.as_default():    
    tf.summary.image(img_name, plot_to_image(figure), step=0)

#### Dynamic images

In [ ]:
train_model_with_cm(x_train, y_train, x_test, y_test, epochs=10)

## TensorBoard Profiler

### Profiler package install

In [ ]:
!pip install -U tensorboard-plugin-profile

### Code

In [ ]:
# Define log folder
log_profiler = 'log_profiler'

In [ ]:
%tensorboard --logdir=log_profiler

In [ ]:
train_model(x_train, y_train, x_test, y_test, log_profiler)

## TensorBoard Debugger

The Debugger V2 GUI has Alerts, Python Execution Timeline, Graph Execution, and Graph Structure. The Alerts section shows your program’s anomalies. The Python Execution Timeline section shows the history of the eager execution of operations and graphs. 

The Graph Execution displays the history of all the floating-dtype tensors that have been computed inside graphs. The Graph Structure section has the Source Code and Stack Trace that are populated as you interact with the GUI. 

### Code

In [ ]:
tf.debugging.experimental.enable_dump_debug_info('logs_debugger', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

In [ ]:
%tensorboard --logdir=logs_debugger

In [ ]:
train_model(x_train[:50], y_train[:50], x_test[:50], y_test[:50], epochs=10, log_folder='logs_debugger')

In [ ]:
!kill 1874

In [ ]:
tf.debugging.experimental.disable_dump_debug_info()